# TF Cascade

In [1]:
import torchvision
import torch
import pandas as pd
import numpy as np
from torchvision import transforms
from PIL import Image
import os

data_folder_path = '/home/cc/object-store/datasets'
dataset_folder_path = os.path.join(
    data_folder_path, 'ILSVRC/Data/DET/test'
)
classes_file_path = os.path.join(
    data_folder_path, 'imagenet_classes.txt'
)

/home/cc/miniconda3/envs/central/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open(classes_file_path) as f:
    classes = [line.strip() for line in f.readlines()]

x = np.array([])
directory = os.fsencode(dataset_folder_path)

for root, dirs, files in os.walk(dataset_folder_path):
    for filename in files:
        x = np.append(x, filename)
df = pd.DataFrame(data=x, columns=["images"])
df['images'][0]

'ILSVRC2017_test_00005500.JPEG'

In [71]:
def resnet_model(img):
    """
    ResNet101 for image classification on ResNet
    """
    # standard resnet image transformation
    transform = transforms.Compose([
    transforms.Resize(256),                    
    transforms.CenterCrop(224), 
    transforms.ToTensor(),                     
    transforms.Normalize(                      
    mean=[0.485, 0.456, 0.406],                
    std=[0.229, 0.224, 0.225]                  
    )])
    
    resnet = torchvision.models.resnet101(pretrained=True)
    resnet.eval()
    # img_2 = Image.open(os.path.join(dataset_folder, img[0]))
    img_t = transform(img['images'])
    batch_t = torch.unsqueeze(img_t, 0)
    out = resnet(batch_t)
    _, indices = torch.sort(out, descending=True)
    percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
    p_2 = percentage.detach().numpy()
    return indices.detach().numpy()[0], p_2, p_2[indices[0][0]]


In [72]:
def inceptionv3_model(img):
    transform = transforms.Compose([
    transforms.Resize(256),                    
    transforms.CenterCrop(224),                
    transforms.ToTensor(),                     
    transforms.Normalize(                      
    mean=[0.485, 0.456, 0.406],                
    std=[0.229, 0.224, 0.225]                  
    )])
    
    resnet = torchvision.models.inception_v3(pretrained=True)
    resnet.eval()
    # img_2 = Image.open(os.path.join(dataset_folder, img[0]))
    img_t = transform(img['images'])
    batch_t = torch.unsqueeze(img_t, 0)
    out = resnet(batch_t)
    _, indices = torch.sort(out, descending=True)
    percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
    p_2 = percentage.detach().numpy()
    return indices.detach().numpy()[0], p_2, p_2[indices[0][0]]

In [73]:
def cascade_predict(row):
    """
    cascade predict based on resnt and compare the final results
    """
    r_index = row[1]
    r_perc = row[2]
    r_max_prob = row[3]
    i_index = row[4]
    i_perc = row[5]
    i_max_prob= row[6]
#     print(np.isnan(i_max_prob))
    
    if np.isnan(i_max_prob):
        # didn't go to inception because resnet prediction was confident enough
        return r_index, r_perc, classes[r_index[0]]
    else:
        #choose the distribution with the higher max_prob
        if r_max_prob > i_max_prob:
            return r_index, r_perc, classes[r_index[0]]
        else:
            return i_index, i_perc, classes[i_index[0]]


In [4]:
def filter_color_images(img):
    img_2 = Image.open(os.path.join(dataset_folder_path, img[0]))
    if img_2.mode == 'RGB':
        return True
    return False

In [5]:
def show(img):
    img_2 = Image.open(os.path.join(dataset_folder_path, img))
    img_2.show()

def load_pics(img):
    img = Image.open(os.path.join(dataset_folder_path, img))
    return img

In [6]:
num_images = 10
# client side preprocess
df = df.sort_values(by=['images'])
# df['image_names'] = df['images']
df_s = df.head(num_images)
df_s = df_s[df_s.apply(filter_color_images, axis=1)]
df_s['images'] = df_s['images'].apply(load_pics)

In [ ]:
img_index = 0
res = resnet_model(df_s.iloc[img_index])
res

In [ ]:
resnet_preds = df_s.apply(
    resnet_model, axis=1, result_type="expand").rename(
        columns={0: "resnet_indices", 1: "resnet_percentage", 2: "resnet_max_prob"})
resnet_preds

In [ ]:
resnet_preds = df_s.apply(
    resnet_model, axis=1, result_type="expand").rename(
        columns={0: "resnet_indices", 1: "resnet_percentage", 2: "resnet_max_prob"}) 

# only send the queries that are above a 85 percnt to the resnet model
# then join it with the original 
inception_preds = df_s.join(
    resnet_preds[resnet_preds['resnet_max_prob'] < 85], how='right').apply(
        inceptionv3_model, axis=1, result_type="expand").rename(
            columns={0: "inception_indices", 1: "inception_percentage", 2:"inception_max_prob"})
inception_preds

In [ ]:
resnet_preds = df_s.apply(
    resnet_model, axis=1, result_type="expand").rename(
        columns={0: "resnet_indices", 1: "resnet_percentage", 2: "resnet_max_prob"}) 

# only send the queries that are above a 85 percnt to the resnet model
# then join it with the original 
inception_preds = df_s.join(
    resnet_preds[resnet_preds['resnet_max_prob'] < 85], how='right').apply(
        inceptionv3_model, axis=1, result_type="expand").rename(
            columns={0: "inception_indices", 1: "inception_percentage", 2:"inception_max_prob"})

all_preds = df_s.join([resnet_preds, inception_preds])
all_preds 



In [ ]:
# Calling the cascading function
cascade_df = all_preds.join(
    all_preds.apply(
        cascade_predict, axis=1, result_type="expand").rename(
            columns={0: "cascade_indices", 1: "cascade_percentage", 2:"cascade_prediction"}))
cascade_df


In [ ]:
cascade_df[['images', 'cascade_prediction']]

In [ ]:
show('ILSVRC2017_test_00000009.JPEG')

In [ ]:
resnet_preds['resnet_indices']

In [ ]:
resnet_preds['resnet_indices']

In [86]:
np.where(resnet_preds['resnet_indices'].iloc[0] == 0)[0][0]

887

In [87]:
np.argmax(resnet_preds['resnet_percentage'].iloc[0])

291

In [88]:
resnet_preds['resnet_percentage'].iloc[0][291]

57.083775